In [107]:
from bs4 import BeautifulSoup
import pandas
import re
import requests

In [159]:
dataframe = pandas.DataFrame(columns=["County", "Contact Name", "Email", "Website", "Facebook Page", "Twitter", "Address", "Phone"])

# Get the URLs of all county contact pages.
req = requests.get("https://www.padems.com/counties/")
pa_counties_page = BeautifulSoup(req.text, "html.parser")
pa_counties = pa_counties_page.find_all(href=re.compile(r'^https://www.padems.com/county'))

# loop over all pa counties.
for pa_county in pa_counties:

    # Reset contact dict.
    county_info = {
        "County": "",
        "Contact Name": "",
        "Email": "",
        "Website": "",
        "Facebook Page": "",
        "Twitter": "",
        "Address": "",
        "Phone": ""
        }

    # get the contact info page for a county.
    req = requests.get(pa_county["href"])
    county = BeautifulSoup(req.text, "html.parser")

    # get the county name.
    county_name = pa_county["href"][30:-1].title()
    county_info["County"] = county_name

    # get the county contacts name.
    person = county.find_all("h3", class_="text-center")
    name = person[0].text
    contact = name[name.rfind("\t")+1:name.rfind(",")]
    county_info["Contact Name"] = contact

    # get the county web links.
    links = county.find(class_="box match")

    for link in links.contents:
        if link.name == "a":
            href = link["href"]
            if "mailto:" in href:
                county_info["Email"] = link["href"][7:]
            elif "facebook" in href.lower():
                county_info["Facebook Page"] = href
            elif "twitter" in href.lower():
                county_info["Twitter"] = href
            else:
                county_info["Website"] = href

    # get county office address.
    address = county.find("address")
    try:
        address = address.text.strip().replace('\n', ' ').replace('\t', '')
        county_info["Address"] = address
    except:
        print(f"No address found for {county_name} county.")

    # county office phone number.
    phone = county.findAll(text=re.compile('\d\d\d-\d\d\d\d$'))
    try:
        phone = phone[0].strip()
        county_info["Phone"] = phone
    except:
        print(f"No phone number found for {county_name} county.")

    # Write county info to a dataframe.
    county_dataFrame = pandas.DataFrame(county_info, index=[0])

    # Append county info to state dataframe.
    dataframe = pandas.concat([dataframe, county_dataFrame], ignore_index=True)


No address found for Beaver county.
No phone number found for Bradford county.
No address found for Cambria county.
No address found for Cameron county.
No phone number found for Clarion county.
No phone number found for Crawford county.
No phone number found for Erie county.
No address found for Forest county.
No phone number found for Franklin county.
No phone number found for Fulton county.
No address found for Greene county.
No phone number found for Greene county.
No phone number found for Huntingdon county.
No address found for Juniata county.
No address found for Lawrence county.
No phone number found for Lawrence county.
No address found for Mckean county.
No phone number found for Mckean county.
No address found for Montour county.
No phone number found for Montour county.
No address found for Northumberland county.
No phone number found for Northumberland county.
No phone number found for Perry county.
No phone number found for Potter county.
No phone number found for Susqueh

In [161]:
dataframe.to_csv("pa_dems.csv", index=False)